# SelectGrid

Powered by [Phosphor DataGrid](https://github.com/phosphorjs/phosphor/blob/master/packages/datagrid/src/datagrid.ts).

In [1]:
import numpy as np, pandas as pd, traitlets as T, ipywidgets as W
from wxyz.datagrid.widget_selectgrid import SelectGrid
from wxyz.lab.widget_dock import DockBox
from wxyz.html.widget_color import AlphaColorPicker

In [2]:
flex = dict(layout=dict(flex="1"))

In [3]:
def make_select_grid(GridClass=SelectGrid, df=None):
    pg = DockBox(layout=dict(height="60vh"))
    dg = pg.dg = GridClass()
    dg2 = pg.dg2 = GridClass()
    dg3 = pg.dg3 = GridClass()
    
    rando = None
    if df is not None:
        dg.value = df
    else:
        with W.Output():
            @W.interact
            def rando(rows=(1, 1000), cols=(1, 1000)):
                dg.value = np.random.rand(rows, cols)
    x, y = [
        [
            W.jslink((w, "value"), (dg, f"scroll_{v}")),
            W.jslink((pg.dg, f"max_{v}"), (w, "max")),
            w,
        ][-1]
        for i, v in enumerate("xy")
        for w in [W.IntSlider(description=f"${v}$", layout=dict(flex="1"))] 
    ]
    
    def gloc(g):
        s = g.selection
        return g.value.iloc[s[2]: s[3]+ 1, s[0]:s[1] + 1]
    
    def _slice(c):
        s = dg.selection
        dg2.value = gloc(dg)
        dg.description = f"{dg2.value.values.sum()}"
        
    
    def _slice2(c):
        s = dg2.selection
        dg3.value = gloc(dg2)
        dg2.description = f"{dg3.value.values.sum()}"
    
    def _slice3(c):
        s = dg3.selection
        v = gloc(dg3)
        dg3.description = f"{v.values.sum()}"
    
    dg.observe(_slice, ["selection", "value"])
    dg2.observe(_slice2, ["selection", "value"])
    dg3.observe(_slice3, ["selection", "value"])
    
    color = AlphaColorPicker(description="select color")
    [T.dlink((color, 'value'), (g, 'selection_color')) for g in [dg, dg2, dg3]]
    
    tabs = W.ToggleButton(description="tabs")
    T.link((tabs, "value"), (pg, "hide_tabs"))
    
    vp = W.Text(description="view")
    T.dlink((pg.dg, "viewport"), (vp, "value"), str)
    
    pg.children = [
        W.VBox([*(rando.widget.children if rando else []), vp, x, y, color, tabs]), 
        dg, 
        dg2,
        dg3
    ]
    pg.dock_layout ={
     'type': 'split-area',
     'orientation': 'horizontal',
     'children': [{'type': 'tab-area', 'widgets': [1], 'currentIndex': 0},
      {'type': 'tab-area', 'widgets': [2], 'currentIndex': 0},
      {'type': 'split-area',
       'orientation': 'vertical',
       'children': [{'type': 'tab-area', 'widgets': [3], 'currentIndex': 0},
        {'type': 'tab-area', 'widgets': [0], 'currentIndex': 0}],
       'sizes': [1, 1]}],
     'sizes': [4, 2, 2]
    }
    return pg

In [4]:
if __name__ == "__main__":
    pg = make_select_grid()
    display(pg)

    pg.dg.viewport = (21, 30, 10, 18)

    i_0 = W.IntSlider(description="$i_0$")
    T.dlink((i_0, "value"), (pg.dg, "viewport"), lambda vc: (vc, *pg.dg.viewport[1:]))

    j_0 = W.IntSlider(description="$j_0$")
    T.dlink((j_0, "value"), (pg.dg, "viewport"), lambda vr: (*pg.dg.viewport[:2], vr, *pg.dg.viewport[3:]))
    display(W.HBox([i_0, j_0]))

DockBox(children=(VBox(children=(IntSlider(value=500, description='rows', max=1000, min=1), IntSlider(value=50…